##### Agent sử dụng tool từ custom function

In [1]:
from configparser import ConfigParser
from langchain.llms import OpenAI
import os

config = ConfigParser()
config.read("../config.ini")
os.environ["OPENAI_API_KEY"]=config["KEY"]["openai_key"]


## Create Tools

In [5]:
from langchain.agents import tool

In [12]:
@tool
def get_word_length(word:str):
        '''Return the length of word'''
        return len(word)

In [13]:
get_word_length.invoke("abc")

3

In [14]:
tools = [get_word_length]

## Create LLM Model

In [18]:
from langchain_openai import ChatOpenAI

In [19]:
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0.2)

## Create Prompt

In [15]:
from langchain import hub

In [16]:
prompt = hub.pull("hwchase17/openai-functions-agent")

/Users/chessman/miniforge3/envs/PyJupyter/lib/python3.10/site-packages/langsmith/client.py:323: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [17]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

## Create Bind Tools

In [25]:
llm_with_tools = llm.bind_tools(tools)

## Create Chain Agent

In [26]:
from langchain.agents import create_tool_calling_agent, AgentExecutor


In [27]:
agent = create_tool_calling_agent(llm=llm_with_tools, tools=tools, prompt=prompt)

## Create Agent Executor

In [28]:
agent_executor = AgentExecutor(agent=agent, tools=tools, prompt=prompt, verbose=True)

In [30]:
list(agent_executor.stream({
        "input":"how many letters in the word AIVIETNAM"
}))



> Entering new None chain...

Invoking: `get_word_length` with `{'word': 'AIVIETNAM'}`


9The word "AIVIETNAM" has 9 letters.

> Finished chain.


[{'actions': [ToolAgentAction(tool='get_word_length', tool_input={'word': 'AIVIETNAM'}, log="\nInvoking: `get_word_length` with `{'word': 'AIVIETNAM'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_jIrUGY7ZiNyPwMUPFIUW3bSO', 'function': {'arguments': '{"word":"AIVIETNAM"}', 'name': 'get_word_length'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-3.5-turbo-0125'}, id='run-f6441e1a-e876-4fba-a245-ff37d141c11b', tool_calls=[{'name': 'get_word_length', 'args': {'word': 'AIVIETNAM'}, 'id': 'call_jIrUGY7ZiNyPwMUPFIUW3bSO', 'type': 'tool_call'}], tool_call_chunks=[{'name': 'get_word_length', 'args': '{"word":"AIVIETNAM"}', 'id': 'call_jIrUGY7ZiNyPwMUPFIUW3bSO', 'index': 0, 'type': 'tool_call_chunk'}])], tool_call_id='call_jIrUGY7ZiNyPwMUPFIUW3bSO')],
  'messages': [AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_jIrUGY7ZiNyPwMUPFIUW3bSO', 'function'

## Create Memory

In [ ]:
from langchain.schema import AIMessage, HumanMessage

### Manual

In [ ]:
agent_executor.invoke(
        {
                "chat_history":[
                        HumanMessage(content="Hello, my name is Eric"),
                        AIMessage(content="Hi Eric, How can I help you")
                ],
                "input":"what is my name"
        }
)



> Entering new AgentExecutor chain...
Your name is Eric.

> Finished chain.


{'chat_history': [HumanMessage(content='Hello, my name is Eric', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hi Eric, How can I help you', additional_kwargs={}, response_metadata={})],
 'input': 'what is my name',
 'output': 'Your name is Eric.'}

### Auto

In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [ ]:
message_history = ChatMessageHistory()

In [ ]:
agent_with_message_history = RunnableWithMessageHistory(
        agent_executor,
        lambda session_id: message_history,
        input_messages_key="input",
        history_messages_key="chat_history"
)

In [ ]:
agent_with_message_history.invoke(
        {"input":"Hi, my name is Red"},
        config={"configurable":{"session_id": "<foo>"}}
)



> Entering new AgentExecutor chain...
Hello Red! How can I assist you today?

> Finished chain.


{'input': 'Hi, my name is Red',
 'chat_history': [],
 'output': 'Hello Red! How can I assist you today?'}

In [ ]:
agent_with_message_history.invoke(
        {"input":"what is my name"},
        config={"configurable":{"session_id": "<foo>"}}
)



> Entering new AgentExecutor chain...
Your name is Red. How can I help you further, Red?

> Finished chain.


{'input': 'what is my name',
 'chat_history': [HumanMessage(content='Hi, my name is Red', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello Red! How can I assist you today?', additional_kwargs={}, response_metadata={})],
 'output': 'Your name is Red. How can I help you further, Red?'}